In [1]:
import sys
sys.path.append('../..')
import os

import matplotlib as mpl
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.utils import to_categorical
from keras.callbacks import TensorBoard

import cr_interface as cri
import keras_utils as ku

Using TensorFlow backend.


In [30]:
'13'.center(23, '-')

'-----------13----------'

In [19]:
app = ku.applications['vgg16']

def load_data(app):
    data = {
        'bottlenecks': dict(),
        'filenames': dict(),
        'labels': dict(),
    }
    
    splits = ['test', 'validation', 'train']
    
    for key, d in data.items():
        for split in splits:
            path = os.path.join(key, app.codename)
            path = os.path.join(path, split)
            d[split] = np.load(path)
    
    return data

data = load_data(app)

In [3]:
def compile_model(model, lr=1.0e-4):
    sgd = optimizers.SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=sgd,
        #optimizer='rmsprop',
        metrics=['accuracy'])
    
def load_top_model(app, compiled=True, lr=1.0e-4):
    model = Sequential()
    model.add(Flatten(input_shape=app.get_model().output_shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))

    if compiled:
        compile_model(model, lr)
    
    return model

def train_model(model, bottlenecks, labels, tensorboard_name=None, epochs=10, batch_size=32):
    if tensorboard_name:
        os.makedirs('boards', exist_ok=True)
        tensorboard = TensorBoard(log_dir="boards/{}".format(tensorboard_name))
        callbacks=[tensorboard]
    else:
        callbacks=None
        
    print('loaded board. now fitting...')
    model.fit(bottlenecks['train'], labels['train'],
                  validation_data=(bottlenecks['validation'], labels['validation']),
                  shuffle=True,
                  batch_size=batch_size,
                  epochs=epochs,
                  callbacks=callbacks)

In [28]:
top_model = load_top_model(app)
train_model(top_model, data['bottlenecks'], data['labels'], epochs=10)

loaded board. now fitting...
Train on 4292 samples, validate on 1086 samples
Epoch 1/10
4292/4292 [==============================] - 7s 2ms/step - loss: 0.0226 - acc: 0.9932 - val_loss: 0.2876 - val_acc: 0.9254
Epoch 2/10
4292/4292 [==============================] - 7s 2ms/step - loss: 0.0235 - acc: 0.9928 - val_loss: 0.2837 - val_acc: 0.9365
Epoch 3/10
4292/4292 [==============================] - 7s 2ms/step - loss: 0.0159 - acc: 0.9944 - val_loss: 0.3007 - val_acc: 0.9263
Epoch 4/10
4292/4292 [==============================] - 7s 2ms/step - loss: 0.0168 - acc: 0.9951 - val_loss: 0.3181 - val_acc: 0.9291
Epoch 5/10
4292/4292 [==============================] - 7s 2ms/step - loss: 0.0155 - acc: 0.9956 - val_loss: 0.3203 - val_acc: 0.9273
Epoch 6/10
4292/4292 [==============================] - 7s 2ms/step - loss: 0.0177 - acc: 0.9951 - val_loss: 0.3137 - val_acc: 0.9309
Epoch 7/10
4292/4292 [==============================] - 7s 2ms/step - loss: 0.0193 - acc: 0.9944 - val_loss: 0.3322 - v

In [ ]:
top_model.save('vgg16_top')

In [20]:
top_model = keras.models.load_model('vgg16_top')

In [21]:
top_model.evaluate(data['bottlenecks']['test'], data['labels']['test'])

526/526 [==============================] - 0s 447us/step


[0.7261983858768478, 0.847908744793881]

In [12]:
batch_size = 16

train_image_generator = app.get_image_data_generator(augment=True).flow_from_directory(
    cri.DATA_DIRS['train'],
    target_size=app.image_size,
    batch_size=batch_size)

validation_image_generator = app.get_image_data_generator(augment=True).flow_from_directory(
    cri.DATA_DIRS['validation'],
    target_size=app.image_size,
    batch_size=batch_size)

test_image_generator = app.get_image_data_generator(augment=False).flow_from_directory(
    cri.DATA_DIRS['test'],
    target_size=app.image_size,
    batch_size=batch_size,
    shuffle=False)

Found 4292 images belonging to 3 classes.
Found 1086 images belonging to 3 classes.
Found 526 images belonging to 3 classes.


In [29]:
model.evaluate_generator(train_image_generator, verbose=1,
                         workers=8, use_multiprocessing=True)

108/269 [===========>..................] - ETA: 15:01

Process ForkPoolWorker-164:
Process ForkPoolWorker-166:
Process ForkPoolWorker-167:
Process ForkPoolWorker-163:
Process ForkPoolWorker-161:
Process ForkPoolWorker-162:
Process ForkPoolWorker-165:
Process ForkPoolWorker-160:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.4_

KeyboardInterrupt: 

In [32]:
model.layers[0].layers

In [26]:
model = Sequential()
model.add(app.get_model())
for layer in model.layers[0].layers[:-2]:
    layer.trainable = False
model.add(top_model)
compile_model(model, lr=1.0e-6)

model.fit_generator(
    train_image_generator,
    epochs=5,
    validation_data=validation_image_generator,
    workers=8,
    shuffle=True,
    use_multiprocessing=True)

Epoch 1/1
 28/269 [==>...........................] - ETA: 23:46 - loss: 0.2968 - acc: 0.9397

Process ForkPoolWorker-131:
Process ForkPoolWorker-138:
Process ForkPoolWorker-139:
Process ForkPoolWorker-132:
Process ForkPoolWorker-137:
Process ForkPoolWorker-129:
Process ForkPoolWorker-141:
Process ForkPoolWorker-142:
Process ForkPoolWorker-136:
Process ForkPoolWorker-143:
Process ForkPoolWorker-140:
Process ForkPoolWorker-134:
Process ForkPoolWorker-135:
Process ForkPoolWorker-130:
Process ForkPoolWorker-128:
Process ForkPoolWorker-133:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.4_2/Frameworks/Python.fra

  File "/usr/local/Cellar/python/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/local/Cellar/python/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/Cellar/python/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/local/Cellar/python/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/local/Cellar/python/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
KeyboardInterrupt
  File "/usr/local/Cellar/python/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 334, in get
    w

Epoch 1/1

  File "/usr/local/Cellar/python/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 